<a href="https://colab.research.google.com/github/xzyaoi/openke-review/blob/master/OpenKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# RDF Preparation
#!wget http://udbms.cs.helsinki.fi/assets/DataSet/Film_dataset.zip
#!unzip Film_dataset.zip
#!mkdir rdf
def convert(filepath):
    entities = []
    relations = []
    trains=[]
    with open(filepath,'r') as f:
        contents = f.readlines()
    for each in contents:
        each_relation = each.replace("\n", "").split("\t")
        entities.append(each_relation[0])
        relations.append(each_relation[1])
        entities.append(each_relation[2])
        trains.append({
            "e1": each_relation[0],
            "e2": each_relation[2],
            "rel": each_relation[1]
        })
    # dedup
    entities = list(set(entities))
    relations = list(set(relations))
    del contents
    # set order and write to file
    
    with open("rdf/entity2id.txt",'w') as f:
        f.write(str(len(entities)))
        f.write("\n")
        for index, each in enumerate(entities):
            f.write(str(each) +"\t"+ str(index) + "\n")
    with open("rdf/relation2id.txt",'w') as f:
        f.write(str(len(relations)))
        f.write("\n")
        for index, each in enumerate(relations):
            f.write(str(each) +"\t"+ str(index) + "\n")
    
    # Index all items in Entities and relations so that we could get them much faster in the near future
    print("Indexing")
    D=dict()
    for i, item in enumerate(entities):
      if item not in D:
        D[item] = i
      else:
        pass
    del entities
    for i, item in enumerate(relations):
      if item not in D:
        D[item] = i
      else:
        pass
    del relations
    print("Finished Indexing")
    # handle trains:
    with open("rdf/train2id.txt",'w') as f:
        f.write(str(len(trains)))
        f.write("\n")
        for index, each in enumerate(trains):
            f.write(str(D[each['e1']]) +"\t"+ str(D[each['e2']])+"\t" +str(D[each['rel']])+ "\n")
convert("Film_dataset/processed_dataset/film.yago.graph")

Indexing
Finished Indexing


In [2]:
!git clone https://github.com/thunlp/OpenKE
!cd OpenKE/openke/ && bash make.sh


Cloning into 'OpenKE'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 1182 (delta 14), reused 3 (delta 0), pack-reused 1151
Receiving objects: 100% (1182/1182), 276.70 MiB | 32.66 MiB/s, done.
Resolving deltas: 100% (567/567), done.


In [0]:
!pip3 install gpumeter


     |████████████████████████████████| 61kB 2.0MB/s 


In [0]:
!mkdir checkpoint
!mkdir result

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TransE on FB15K-237
import sys
import os
sys.path.append("./OpenKE")
import openke
from gpumeter import Meter
from openke.config import Trainer, Tester
from openke.module.model import TransE
from openke.module.loss import MarginLoss
from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader

# dataloader for training
train_dataloader = TrainDataLoader(
	in_path = "./OpenKE/benchmarks/FB15K237/", 
	nbatches = 100,
	threads = 8, 
	sampling_mode = "normal", 
	bern_flag = 1, 
	filter_flag = 1, 
	neg_ent = 25,
	neg_rel = 0)

# dataloader for test
test_dataloader = TestDataLoader("./OpenKE/benchmarks/FB15K237/", "link")

# define the model
transe = TransE(
	ent_tot = train_dataloader.get_ent_tot(),
	rel_tot = train_dataloader.get_rel_tot(),
	dim = 50, #200,150,100，50
	p_norm = 1, 
	norm_flag = True)


# define the loss function
model = NegativeSampling(
	model = transe, 
	loss = MarginLoss(margin = 5.0),
	batch_size = train_dataloader.get_batch_size()
)


# train the model
m = Meter(20)
trainer = Trainer(model = model, data_loader = train_dataloader, train_times = 500, alpha = 1.0, use_gpu = True) #train time = 1000,800，500
trainer.run()

transe.save_checkpoint('./drive/My Drive/results/transe-d50-t500.ckpt')

# test the model
transe.load_checkpoint('./drive/My Drive/results/transe-d50-t500.ckpt')
tester = Tester(model = transe, data_loader = test_dataloader, use_gpu = True)
tester.run_link_prediction(type_constrain = False)
print("acc&threshlod: ")
print(tester.run_triple_classification())
m.stop()


  0%|          | 0/500 [00:00<?, ?it/s]

Finish initializing...



Epoch 0 | loss: 199.376118:   0%|          | 0/500 [00:00<?, ?it/s]
Epoch 0 | loss: 199.376118:   0%|          | 1/500 [00:00<07:36,  1.09it/s]
Epoch 1 | loss: 79.436617:   0%|          | 1/500 [00:01<07:36,  1.09it/s] 
Epoch 1 | loss: 79.436617:   0%|          | 2/500 [00:01<07:36,  1.09it/s]
Epoch 2 | loss: 53.309776:   0%|          | 2/500 [00:02<07:36,  1.09it/s]
Epoch 2 | loss: 53.309776:   1%|          | 3/500 [00:02<07:35,  1.09it/s]
Epoch 3 | loss: 43.058043:   1%|          | 3/500 [00:03<07:35,  1.09it/s]
Epoch 3 | loss: 43.058043:   1%|          | 4/500 [00:03<07:33,  1.09it/s]
Epoch 4 | loss: 38.152309:   1%|          | 4/500 [00:04<07:33,  1.09it/s]
Epoch 4 | loss: 38.152309:   1%|          | 5/500 [00:04<07:32,  1.09it/s]
Epoch 5 | loss: 34.954707:   1%|          | 5/500 [00:05<07:32,  1.09it/s]
Epoch 5 | loss: 34.954707:   1%|          | 6/500 [00:05<07:33,  1.09it/s]
Epoch 6 | loss: 33.091412:   1%|          | 6/500 [00:06<07:33,  1.09it/s]
Epoch 6 | loss: 33.091412:   

0.37757745385169983
acc&threshlod: 
(0.945299521157041, 5.9426422119140625)
Total Consumed: 10.78 Wh
Your Badge is Ready! See https://img.shields.io/badge/Power%20Consumption-10.78%20Wh-green


10.779883333333332

In [6]:
import sys
sys.path.append("/content/OpenKE/")
print(sys.path)

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/OpenKE/']


In [0]:
# TransR on FB15K-237

import openke
from openke.config import Trainer, Tester
from openke.module.model import TransE, TransR
from openke.module.loss import MarginLoss
from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader
for dim in [100, 150]:
# dataloader for training
	train_dataloader = TrainDataLoader(
		in_path = "./OpenKE/benchmarks/FB15K237/", 
		nbatches = 500,
		threads = 8, 
		sampling_mode = "normal", 
		bern_flag = 1, 
		filter_flag = 1, 
		neg_ent = 25,
		neg_rel = 0)

	# dataloader for test
	test_dataloader = TestDataLoader(
		in_path = "./OpenKE/benchmarks/FB15K237/",
		sampling_mode = 'link')

	# define the model
	transe = TransE(
		ent_tot = train_dataloader.get_ent_tot(),
		rel_tot = train_dataloader.get_rel_tot(),
		dim = dim, 
		p_norm = 1, 
		norm_flag = True)

	model_e = NegativeSampling(
		model = transe, 
		loss = MarginLoss(margin = 5.0),
		batch_size = train_dataloader.get_batch_size())

	transr = TransR(
		ent_tot = train_dataloader.get_ent_tot(),
		rel_tot = train_dataloader.get_rel_tot(),
		dim_e = dim, #150,100
		dim_r = dim, #150
		p_norm = 1, 
		norm_flag = True,
		rand_init = False)

	model_r = NegativeSampling(
		model = transr,
		loss = MarginLoss(margin = 4.0),
		batch_size = train_dataloader.get_batch_size()
	)

	# pretrain transe
	trainer = Trainer(model = model_e, data_loader = train_dataloader, train_times = 1, alpha = 0.5, use_gpu = False)
	trainer.run()

	# transe.load_checkpoint('./checkpoint/transe.ckpt')
	parameters = transe.get_parameters()
	transe.save_parameters("./drive/My Drive/results/transr_transe.json")

	print("current dim"+str(dim))
	# train transr
	transr.set_parameters(parameters)
	trainer = Trainer(model = model_r, data_loader = train_dataloader, train_times = 1000, alpha = 1.0, use_gpu = True)#trian time=2000
	trainer.run()
	transr.save_checkpoint('./drive/My Drive/results//transr-d'+str(dim)+'-t1000.ckpt')

	# test the model
	transr.load_checkpoint('./drive/My Drive/results/transr-d'+str(dim)+'-t1000.ckpt')
	tester = Tester(model = transr, data_loader = test_dataloader, use_gpu = True)
	tester.run_link_prediction(type_constrain = False)

  0%|          | 0/1 [00:00<?, ?it/s]

Finish initializing...


Epoch 0 | loss: 652.721777: 100%|██████████| 1/1 [01:17<00:00, 77.64s/it]


current dim100


  0%|          | 0/1000 [00:00<?, ?it/s]

Finish initializing...


Epoch 13 | loss: 16.446410:   1%|▏         | 14/1000 [02:44<3:13:19, 11.76s/it]

In [0]:
# DistMult on FB15K-237

import openke
from openke.config import Trainer, Tester
from openke.module.model import DistMult
from openke.module.loss import SoftplusLoss
from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader

# dataloader for training
train_dataloader = TrainDataLoader(
	in_path = "./OpenKE/benchmarks/FB15K237/", 
	nbatches = 500,
	threads = 8, 
	sampling_mode = "normal", 
	bern_flag = 1, 
	filter_flag = 1, 
	neg_ent = 25,
	neg_rel = 0)

# dataloader for test
test_dataloader = TestDataLoader(
	in_path = "./OpenKE/benchmarks/FB15K237/",
	sampling_mode = 'link')


# define the model
distmult = DistMult(
	ent_tot = train_dataloader.get_ent_tot(),
	rel_tot = train_dataloader.get_rel_tot(),
	dim = 200 #200
)

# define the loss function
model = NegativeSampling(
	model = distmult, 
	loss = SoftplusLoss(),
	batch_size = train_dataloader.get_batch_size(), 
	regul_rate = 1.0
)


# train the model
trainer = Trainer(model = model, data_loader = train_dataloader, train_times = 1000, alpha = 0.5, use_gpu = True, opt_method = "adagrad")#train time=2000
trainer.run()
distmult.save_checkpoint('./drive/My Drive/results/distmult-d200-t1000.ckpt')

# test the model
distmult.load_checkpoint('./drive/My Drive/results/distmult-d200-t1000.ckpt')
tester = Tester(model = distmult, data_loader = test_dataloader, use_gpu = True)
tester.run_link_prediction(type_constrain = False)
print("acc&threshlod: ")
print(tester.run_triple_classification())






  0%|          | 0/1000 [00:00<?, ?it/s]

Finish initializing...






Epoch 0 | loss: 501.297161:   0%|          | 0/1000 [00:02<?, ?it/s]



Epoch 0 | loss: 501.297161:   0%|          | 1/1000 [00:02<48:49,  2.93s/it]



Epoch 1 | loss: 186.761024:   0%|          | 1/1000 [00:05<48:49,  2.93s/it]



Epoch 1 | loss: 186.761024:   0%|          | 2/1000 [00:05<48:41,  2.93s/it]



Epoch 2 | loss: 131.817553:   0%|          | 2/1000 [00:08<48:41,  2.93s/it]



Epoch 2 | loss: 131.817553:   0%|          | 3/1000 [00:08<48:36,  2.92s/it]



Epoch 3 | loss: 109.108599:   0%|          | 3/1000 [00:11<48:36,  2.92s/it]



Epoch 3 | loss: 109.108599:   0%|          | 4/1000 [00:11<48:32,  2.92s/it]



Epoch 4 | loss: 96.582844:   0%|          | 4/1000 [00:14<48:32,  2.92s/it] 



Epoch 4 | loss: 96.582844:   0%|          | 5/1000 [00:14<48:31,  2.93s/it]



Epoch 5 | loss: 88.784846:   0%|          | 5/1000 [00:17<48:31,  2.93s/it]



Epoch 5 | loss: 88.784846:   1%|          | 6/1000 [00:17<48:29,  2.93s/it]



Epoch 6 | loss: 83.228991:   1%|          | 6/1